In [1]:
!module load python3/3.10.10-01
import lmod
await lmod.load('ecmwf-toolbox/2024.04.0.0')
!ml
import metview as mv
import os
import numpy as np
import sys
import matplotlib.pyplot as plt


The following have been reloaded with a version change:
  1) python3/3.11.10-01 => python3/3.10.10-01


Currently Loaded Modules:
  1) gcc/8.5.0    3) python3/3.11.10-01   5) ecmwf-toolbox/2024.04.0.0
  2) prgenv/gnu   4) node/20.11.0

 



In [2]:
# general fucntion to do the plot
def mv_ql_plot(var2plotIn, latIncIn, lonIncIn, latlonMapIn, 
               contourMinIn, contourMaxIn, contourListIn, txttitleIn,plotname=None):
    
    coastlines = mv.mcoast(
        map_coastline_colour            = "black",
        map_coastline_thickness         = 3.5,
        map_coastline_resolution        = "full",
        map_coastline_land_shade        = "on",
        map_coastline_land_shade_colour = "RGB(0.8627,0.8627,0.8627)",
        map_coastline_sea_shade         = "on",
        map_coastline_sea_shade_colour  = "RGB(0.92,0.94,0.93)",
        map_boundaries                  = "on",
        map_rivers                      = "off",
        map_boundaries_colour           = "black",
        map_boundaries_thickness        = 2.5,
        map_grid_colour                 = "grey",
        map_label_colour                = "RGB(0,0,0)",
        map_grid_latitude_increment     = latIncIn,    
        map_grid_longitude_increment    = lonIncIn
    )
    
    view = mv.geoview(
        map_projection="cylindrical",
        map_area_definition = "corners",
        area                = latlonMapIn,    
        coastlines          = coastlines,
        )
    
    xs_shade = mv.mcont(
            legend="on",
        contour="off",
        contour_level_selection_type = "level_list",
        contour_level_list = contourListIn,
        contour_label="off",
        contour_shade="on",
        contour_shade_colour_method ="list",
        contour_hilo                     = "on",
        contour_lo_colour                = "black",
        contour_hi_colour                = "black",
        contour_lo_min_value             = +1.0E21,
        contour_hi_min_value             = 75,
        contour_hilo_quality             = "high",
        contour_hilo_type                = "number",
        contour_hilo_format              = "(I5)",
        contour_shade_method             = "area_fill",
        contour_shade_colour_list=["RGB(0.75,0.95,0.93)","RGB(0.45,0.93,0.78)","RGB(0.06999,0.85,0.61)","RGB(0.53,0.8,0.13)","RGB(0.6,0.91,0.05699)","RGB(0.9,1,0.4)","RGB(0.89,0.89,0.066)","RGB(1,0.73,0.003906)","RGB(1,0.49,0.003906)","red","RGB(0.85,0.003906,1)","RGB(0.63,0.007294,0.92)","RGB(0.03999,0.03999,0.84)","RGB(0.04199,0.04199,0.43)","RGB(0.45,0.45,0.45)"]
        #contour_shade_colour_list=["RGB(0.75,0.95,0.93)","RGB(0.45,0.93,0.78)","RGB(0.06999,0.85,0.61)","RGB(0.53,0.8,0.13)","RGB(0.6,0.91,0.05699)","RGB(0.9,1,0.4)","RGB(0.89,0.89,0.066)","RGB(1,0.73,0.003906)","RGB(1,0.49,0.003906)","red","RGB(0.85,0.003906,1)","RGB(0.63,0.007294,0.92)","RGB(0.03999,0.02999,0.84)","white","RGB(0.45,0.45,0.45)"]

    )
    
    legend = mv.mlegend(
        legend_text_font_size = 0.4,
    )
    
    title = mv.mtext(text_line_1 = txttitleIn,text_font_size=0.8)
    
    mv.plot(var2plotIn, 
            xs_shade,
            coastlines,
            view,
            legend,
            title)

    if plotname is not None:
        #plotname = os.path.join(pathDiagPlots, myPng)    
        #plt.savefig(plotname)
        mv.setoutput(mv.png_output(output_name=plotname,output_width=1200))
            
    return None

**CONFIGURE CASE**

In [3]:
# map set up
# your lat/lon domain [SE lat, SE lon, NW lat, NW lon)
#lat_lon_map_area = [53,1,58,20]    
lat_lon_map_area = [52,4,59,18]    

# interval between lat/lon
lat_inc = 2
lon_inc = 2

# colors
contour_shade_min_level_colour = "blue"    
contour_shade_max_level_colour = "magenta"

# intervals 
contour_level_list = [0.5, 1, 3, 5, 7, 10, 15, 20, 25, 30, 40, 50, 60, 70] #,125,150,200,300,500]
#[1, 2, 3, 4, 6, 8, 10, 12, 14, 16, 18, 24]  

topPath = '/perm/miag/deode_exps/2024121400'
validTime = '20241215-12UTC'
fc_setp2 = '36'
fc_setp1 = '30'

savePlot = False

if not (savePlot):
    mv.setoutput('jupyter')

In [4]:
##########
# DEODE EXP
##########
gribId = 'GRIBPFDEOD'
strExp = 'CY46h1_HARMONIE_AROME_DAN_1500x1500_500m_v1'
txttitle = '6h accumulated 500m Harmonie-Arome [valid time: ' + validTime + ']'
plotName = 'acc_prep.harmonie.validTime.'+validTime+'.png'
####################
gbFile  = os.path.join(topPath,strExp,'surface_{0}+00{1}h00m00s'.format(gribId,fc_setp1))
gbFile2 = os.path.join(topPath,strExp,'surface_{0}+00{1}h00m00s'.format(gribId,fc_setp2))

# Reading the grib
myFC = mv.read(gbFile) 
# This is nececssary only for the DEODE grib file to replace gridType = lambert_lam with gridType = lambert 
myFC = mv.grib_set(myFC,["gridType",'lambert'])

tirf1  = myFC.select(shortName="tirf") 
tsnow1 = myFC.select(shortName="tsnowp") 
tgrp1  = myFC.select(shortName="tgrp") 

tot1 = tirf1 + tsnow1 + tgrp1

myFC2 = mv.read(gbFile2) 
myFC2 = mv.grib_set(myFC2,["gridType",'lambert'])

tirf2  = myFC2.select(shortName="tirf") 
tsnow2 = myFC2.select(shortName="tsnowp") 
tgrp2  = myFC2.select(shortName="tgrp") 

tot2 = tirf2 + tsnow2 + tgrp2

acc_prep = tot2 - tot1

#check min max
myMin = int(np.ceil(mv.minvalue(acc_prep)))    
myMax = int(np.floor(mv.maxvalue(acc_prep)))

myMin, myMax

if (savePlot):
    mv.setoutput(mv.png_output(output_name=plotName,output_width=1200))

mv_ql_plot(acc_prep, lat_inc, lon_inc, lat_lon_map_area, 
           contour_shade_min_level_colour, contour_shade_max_level_colour, contour_level_list, txttitle)



Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

In [5]:
##########
# DT
##########
gribId = '2024121400'
strExp = 'DT'
txttitle = '6h accumulated DT [valid time: ' + validTime + ']'
plotNameDT = 'acc_prep.DT.validTime.'+validTime+'.png'
####################
gbFile  = os.path.join(topPath,strExp,'DT_{0}T{1}.grb'.format(gribId,fc_setp1))
gbFile2 = os.path.join(topPath,strExp,'DT_{0}T{1}.grb'.format(gribId,fc_setp2))

# Reading the grib
myFC = mv.read(gbFile) 
myFC2 = mv.read(gbFile2) 

tirf2 = myFC2.select(shortName="tp") 
tirf1 = myFC.select(shortName="tp")  

acc_prep = (tirf2 - tirf1)*1000.0

#check min max
myMin = int(np.ceil(mv.minvalue(acc_prep)))    
myMax = int(np.floor(mv.maxvalue(acc_prep)))

myMin, myMax

if (savePlot):
    mv.setoutput(mv.png_output(output_name=plotNameDT,output_width=1200))

mv_ql_plot(acc_prep, lat_inc, lon_inc, lat_lon_map_area, 
           contour_shade_min_level_colour, contour_shade_max_level_colour, contour_level_list, txttitle) #,plotname=plotName)



Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')

In [6]:
##########
# HRES
##########
gribId = '2024121400'
strExp = 'HRES'
txttitle = '6h accumulated HRES [valid time: ' + validTime + ']'
plotNameHRES = 'acc_prep.HRES.validTime.'+validTime+'.png'
####################
gbFile  = os.path.join(topPath,strExp,'HRES_{0}T{1}.grb'.format(gribId,fc_setp1))
gbFile2 = os.path.join(topPath,strExp,'HRES_{0}T{1}.grb'.format(gribId,fc_setp2))

# Reading the grib
myFC = mv.read(gbFile) 
myFC2 = mv.read(gbFile2) 

tirf2 = myFC2.select(shortName="tp") 
tirf1 = myFC.select(shortName="tp")  

acc_prep = (tirf2 - tirf1)*1000.0

#check min max
myMin = int(np.ceil(mv.minvalue(acc_prep)))    
myMax = int(np.floor(mv.maxvalue(acc_prep)))

myMin, myMax

if (savePlot):
    mv.setoutput(mv.png_output(output_name=plotNameHRES,output_width=1200))

mv_ql_plot(acc_prep, lat_inc, lon_inc, lat_lon_map_area, 
           contour_shade_min_level_colour, contour_shade_max_level_colour, contour_level_list, txttitle) #,plotname=plotName)


Image(value=b'', layout="Layout(visibility='hidden')")

Label(value='Generating plots....')